In [29]:
"""
The purpose of this file is to take in the entire data for Ann Arbor Airport, resample and scale it to 0->1 range
and output it as a .csv file for the other notebooks to process further. 

The feature analysis notebook will extract the most meaningful features and remove multicollinearity

The data was collected via Climate data online, NOAA. We manually entered the power outage data in. 
"""
import pandas as pd
import pandas_ml as pdml
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA


In [30]:
#Load data
df = pd.read_csv('data.csv');
#Replace NaN with 0
df = df.fillna(0);
df['DATE'] = pd.to_datetime(df['DATE']);

In [31]:
#Add columns for wind squared and reorder the tables
df['Avg Wind Speed Squared'] = pd.Series(np.square(df['Average Wind Speed']), index=df.index)
df['5 second wind speed squared'] = pd.Series(np.square(df['Fastest 5 second wind speed']));
df['2 min wind speed squared'] = pd.Series(np.square(df['Fastest 2 minute wind speed']));
df = df[['Power Outage', 'PRCP', 'TMAX','TMIN','Fastest 5 second wind speed','5 second wind speed squared', 'Fastest 2 minute wind speed' ,'2 min wind speed squared','Average Wind Speed','Avg Wind Speed Squared','Fog/Ice', 'Heavy/Freezing Fog', 'Thunder','Smoke/Haze']];


In [32]:
#Under sample based on Cluster Centroids
df_ML = pdml.ModelFrame(df, target = 'Power Outage')
df_ML.target_name
print("Initial Counts");
print(df_ML.target.value_counts());
sampler = df_ML.imbalance.under_sampling.ClusterCentroids();
sampled = df_ML.fit_sample(sampler);
print("After sampling Counts:")
print(sampled.target.value_counts());
finalData = pd.DataFrame(sampled);
finalData.head()

Initial Counts
0.0    2751
1.0     159
Name: Power Outage, dtype: int64
After sampling Counts:
1.0    159
0.0    159
Name: Power Outage, dtype: int64


,Power Outage,PRCP,TMAX,TMIN,Fastest 5 second wind speed,5 second wind speed squared,Fastest 2 minute wind speed,2 min wind speed squared,Average Wind Speed,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
0,0.0,0.000000,85.000000,53.000000,83.000000,6889.000000,32.000000,1024.000000,2.910000,8.468100,1.000000,1.000000,0.000000,1.000000
1,0.0,0.087143,45.028571,26.228571,29.357143,862.007143,21.720000,471.888000,10.692857,115.096254,0.342857,0.028571,0.028571,0.228571
2,0.0,0.000000,52.000000,35.000000,172.900000,29894.410000,36.900000,1361.610000,15.880000,252.174400,0.000000,0.000000,0.000000,0.000000
3,0.0,0.460000,62.000000,38.000000,136.000000,18496.000000,21.900000,479.610000,9.400000,88.360000,1.000000,0.000000,0.000000,0.000000
4,0.0,0.054737,75.263158,49.084211,21.483158,461.727474,15.868421,252.254211,5.952632,37.603141,0.326316,0.157895,0.052632,0.105263


In [33]:
#This was our initial approach, but it gave substantially less accuracy
# Random Split
#Split the data in two parts.
# powerOutages = df.loc[df['Power Outage'] == 1];
# noPowerOutages = df.loc[df['Power Outage'] == 0];

# zeroDays = noPowerOutages.sample(159);
# finalData = pd.concat([powerOutages,zeroDays]);
# finalData = finalData.sample(318);


In [34]:
#Scale down the data to a 0->1 range
scaledData = preprocessing.MinMaxScaler().fit_transform(finalData.values);
finalData = pd.DataFrame(scaledData, index = finalData.index, columns = finalData.columns);

In [35]:
finalData.to_csv('Clustered Data', index = False);